In [62]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adadelta
from sklearn.model_selection import train_test_split
import pandas as pd


In [55]:
X = np.arange(1, 50)
Y = X * 15
print("Sequence:", X, Y, sep="\n")
X = X.reshape(-1, 1, 1)
Y = Y.reshape(-1, 1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, shuffle=False)

Sequence:
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49]
[ 15  30  45  60  75  90 105 120 135 150 165 180 195 210 225 240 255 270
 285 300 315 330 345 360 375 390 405 420 435 450 465 480 495 510 525 540
 555 570 585 600 615 630 645 660 675 690 705 720 735]


In [56]:
def get_model():
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(1, 1)))
    model.add(Dense(1))
    model.compile(optimizer=Adadelta(learning_rate=0.03), loss='mse')
    return model

In [57]:
tf.random.set_seed(0)


In [58]:
model = get_model()
print(model.summary())


C:\Users\karth\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_10 (LSTM)                  │ (None, 50)             │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,451 (40.82 KB)

 Trainable params: 10,451 (40.82 KB)

 Non-trainable params: 0 (0.00 B)

None


In [59]:
model.fit(X_train, Y_train, epochs=2000, validation_split=0.2, batch_size=10)


Epoch 1/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - loss: 58112.9102 - val_loss: 240737.2188
Epoch 2/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 58109.7109 - val_loss: 240723.5312
Epoch 3/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 58106.5781 - val_loss: 240709.9062
Epoch 4/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 58103.4570 - val_loss: 240696.2812
Epoch 5/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 58100.3359 - val_loss: 240682.6719
Epoch 6/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 58097.2109 - val_loss: 240669.0312
Epoch 7/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 58094.0859 - val_loss: 240655.3750
Epoch 8/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 58090.9531 - val_loss: 240641.7344
Epoch 9/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 58087.8203 - val_loss: 240628.0625
Epoch 10/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 58084.6875 - val_loss: 240614.4062
Epoch 11/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss:

In [67]:
tf.keras.models.save_model(model, "./models/seq_pred_model.keras")
model = tf.keras.models.load_model("./models/seq_pred_model.keras")

C:\Users\karth\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 7 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [68]:
Y_pred = model.predict(X_test, verbose=0)
Y_diff = Y_test - Y_pred
df = pd.DataFrame(np.hstack([Y_test, Y_pred, Y_diff]), columns=["Actual", "Predicted", "Difference"])
print(df)

    Actual   Predicted  Difference
0    555.0  553.573792    1.426208
1    570.0  568.425781    1.574219
2    585.0  583.273315    1.726685
3    600.0  598.116394    1.883606
4    615.0  612.955078    2.044922
5    630.0  627.789734    2.210266
6    645.0  642.620178    2.379822
7    660.0  657.446716    2.553284
8    675.0  672.269287    2.730713
9    690.0  687.088135    2.911865
10   705.0  701.903137    3.096863
11   720.0  716.714600    3.285400
12   735.0  731.522400    3.477600
